In [1]:
from pip_flow.pip_flow import PipFlow


pip_flow=PipFlow(device='cloud', model_key="PipableAI/pipable-worflow-automaton-private-v2")

In [2]:
prompt="""
<question>
write a python function to double the input number.
</question>
<code>
"""
pip_flow.generate(prompt, eos_token="code")

'\n\n```python\ndef double_number(num):\n  return num * 2\n```\n\n'

In [3]:
import requests
from bs4 import BeautifulSoup

functions = [
requests.get,
BeautifulSoup,
BeautifulSoup.find_all,
]
docs= pip_flow.generate_docs(requests.post)
print(docs)
pip_flow.register_callables(functions,generate_docs=False) #keeps on appending to the object


The function `post` sends a POST request to the specified URL. It takes several parameters including `url` for the URL of the site to be accessed, `data` for sending data in the body of the POST request, `json` for sending JSON data, and `**kwargs` for any other parameters that the `requests.post()` function might take.

The function returns a `Response` object from the `requests` module.

**Parameters**

- `url`: The URL of the site to be accessed. It should be a string.
- `data`: The data to be sent in the body of the POST request. It can be a dictionary, list of tuples, bytes, or file-like object. The default value is `None`.
- `json`: A JSON serializable Python object to be sent in the body of the POST request. It can be a dictionary, list, tuple, string, int, float, bool, or None. The default value is `None`.
- `**kwargs`: Any other parameters that the `requests.post()` function might take.

**Return**

- A `Response` object from the `requests` module.

**Dtypes**

- `url`: A str

In [4]:
pip_flow.functions

[Function(signature='(url, params=None, **kwargs)', docs='Sends a GET request.\n\n    :param url: URL for the new :class:`Request` object.\n    :param params: (optional) Dictionary, list of tuples or bytes to send\n        in the query string for the :class:`Request`.\n    :param \\*\\*kwargs: Optional arguments that ``request`` takes.\n    :return: :class:`Response <Response>` object\n    :rtype: requests.Response\n    ', name='get', full_name='requests.api.get'),
 Function(signature="(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)", docs='A data structure representing a parsed HTML or XML document.\n\n    Most of the methods you\'ll call on a BeautifulSoup object are inherited from\n    PageElement or Tag.\n\n    Internally, this class defines the basic interface called by the\n    tree builders when converting an HTML/XML document into a data\n    structure. The interface abstracts away the differe

In [5]:
prompt = """
<functions>
{func_info}
</functions>
<json_structure>
{{
  "tasks": [
    {{
      "task_id": 1,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }},
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }}
      ],
      "outputs": ["variable_1"],
      "description": "some description"
    }},
    {{
      "task_id": 2,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }},
        {{,
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }}
      ],
      "outputs": ["variable_2"],
      "description": "some description"
    }}
  ]
}}
</json_structure>
<instructions>
- use self parameter with proper value based on the question.
- name outputs as variable_1 , variable_2 , variable_3 , variable_4 and more variables in chronological order.
- give attention to the type annotation of the parameter given while filling values.
{instructions}
</instructions>
<question>
Given the above functions,
- Do not give the parameters in json which have null values and default values of the function, only give the sequencial function calls with parameters to execute the below question:
{question}
</question>
<json>
"""
pip_flow.add_plan_template("default", prompt)

In [6]:

ques = "get the content of https://google.com, create beautiful soup class and find all divs in it"
config = {
"func_info":  str([
                f"""--name:{function.name}\n--annotations:{function.signature}\n--doc:{function.docs}\n\n"""
                for function in pip_flow.functions
            ]),
"instructions": "",
"question": ques
}
print(pip_flow.make_live_prompt("default", config))


<functions>
['--name:get\n--annotations:(url, params=None, **kwargs)\n--doc:Sends a GET request.\n\n    :param url: URL for the new :class:`Request` object.\n    :param params: (optional) Dictionary, list of tuples or bytes to send\n        in the query string for the :class:`Request`.\n    :param \\*\\*kwargs: Optional arguments that ``request`` takes.\n    :return: :class:`Response <Response>` object\n    :rtype: requests.Response\n    \n\n', '--name:BeautifulSoup\n--annotations:(markup=\'\', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)\n--doc:A data structure representing a parsed HTML or XML document.\n\n    Most of the methods you\'ll call on a BeautifulSoup object are inherited from\n    PageElement or Tag.\n\n    Internally, this class defines the basic interface called by the\n    tree builders when converting an HTML/XML document into a data\n    structure. The interface abstracts away the difference

In [7]:
plan =pip_flow.generate_plan()
print(plan)

{
    "tasks": [
        {
            "task_id": 1,
            "function_name": "get",
            "parameters": [
                {
                    "name": "url",
                    "value": "https://google.com",
                    "description": "URL to be fetched",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_1"
            ],
            "description": "Fetches the content of the specified URL using the GET method."
        },
        {
            "task_id": 2,
            "function_name": "BeautifulSoup",
            "parameters": [
                {
                    "name": "markup",
                    "value": "variable_1",
                    "description": "Markup to be parsed",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_2"
            ],
            "description": "Creates a BeautifulSoup object from the parse

In [8]:
pip_flow.plan_to_code()


```python
import requests
from bs4 import BeautifulSoup

# Fetch the content of the URL
response = requests.api.get('https://google.com')

# Create BeautifulSoup object from the fetched content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all div elements in the BeautifulSoup object
divs = soup.find_all('div')

# Print the div elements
for div in divs:
    print(div)
```
